In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from image_graph_util import img_conversion_v7
from sklearn.preprocessing import MinMaxScaler
import os
import glob
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split
from emnist import extract_training_samples,extract_test_samples
from emnist import list_datasets
from sklearn.utils import shuffle
import cv2
import glob

2023-07-24 12:32:19.520901: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-07-24 12:32:19.520935: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
img=img_conversion_v7.image_convert()


patch=2


In [3]:
ds, info = tfds.load('CatsVsDogs', split='train', shuffle_files=True, with_info=True)

2023-07-24 12:32:57.649055: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /data/user/home/sahaj432/.local/lib/python3.9/site-packages/cv2/../../lib64:/cm/shared/apps/slurm/18.08.9/lib64/slurm:/cm/shared/apps/slurm/18.08.9/lib64
2023-07-24 12:32:57.661997: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-07-24 12:32:57.662041: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c0183): /proc/driver/nvidia/version does not exist
2023-07-24 12:32:57.662855: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other opera

In [4]:
train_images = []
train_labels = []

for example in ds:
    train_images.append(example['image'].numpy())
    train_labels.append(example['label'].numpy())

Corrupt JPEG data: 99 extraneous bytes before marker 0xd9
Corrupt JPEG data: 396 extraneous bytes before marker 0xd9
Corrupt JPEG data: 162 extraneous bytes before marker 0xd9
Corrupt JPEG data: 65 extraneous bytes before marker 0xd9
Corrupt JPEG data: 252 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1403 extraneous bytes before marker 0xd9
Corrupt JPEG data: 214 extraneous bytes before marker 0xd9
Corrupt JPEG data: 2226 extraneous bytes before marker 0xd9
Corrupt JPEG data: 128 extraneous bytes before marker 0xd9
Corrupt JPEG data: 239 extraneous bytes before marker 0xd9
Corrupt JPEG data: 1153 extraneous bytes before marker 0xd9
Corrupt JPEG data: 228 extraneous bytes before marker 0xd9


In [5]:
print(len(train_images))
print(len(train_labels))

23262
23262


In [6]:
images=np.array([cv2.resize(i,(100,100), interpolation= cv2.INTER_CUBIC) for i in train_images])


In [7]:
images = images/255
labels = np.array(train_labels)

In [8]:
images=np.array(images,dtype="float16")

In [9]:
images.shape

(23262, 100, 100, 3)

In [10]:
train,test,train_lab,test_lab=train_test_split(images,labels,test_size=0.30,random_state=42)


In [11]:
print(train.shape)
print(test.shape)
print(train_lab.shape)
print(test_lab.shape)

(16283, 100, 100, 3)
(6979, 100, 100, 3)
(16283,)
(6979,)


In [12]:
regular_train=train
reg_train_label=train_lab

In [13]:
regular_val,regular_test,reg_val_label,reg_test_label=train_test_split(test,test_lab,test_size=0.50,random_state=42)


In [14]:
print(regular_train.shape)
print(regular_val.shape)
print(regular_test.shape)



(16283, 100, 100, 3)
(3489, 100, 100, 3)
(3490, 100, 100, 3)


In [15]:
reg_train_patch=img.to_patcharray(regular_train[:],patch_size=(patch,patch))
reg_val_patch=img.to_patcharray(regular_val[:],patch_size=(patch,patch))
reg_test_patch=img.to_patcharray(regular_test[:],patch_size=(patch,patch))


In [16]:
print(reg_train_patch.shape)
print(reg_val_patch.shape)
print(reg_test_patch.shape)

(16283, 3, 2500, 2, 2)
(3489, 3, 2500, 2, 2)
(3490, 3, 2500, 2, 2)


In [17]:
from collections import Counter
#print((Counter(y_val)))
print(np.sort(list(Counter(reg_train_label).keys())))
classes=len(np.sort(list(Counter(reg_train_label).keys())))

[0 1]


In [18]:
reg_train_patch.shape[1]

3

In [19]:
import networkx as nx


def compute_grid_lap_channel(df_in): #compute_grid_lap_channel
    g = nx.generators.lattice.grid_2d_graph(patch,patch, periodic=False)
    adj_ary=nx.adjacency_matrix(g).toarray()

    edge=np.array(np.where(adj_ary==1))
    ed0=edge[0]
    ed1=edge[1]

    no_img,no_ch, no_patch,patch_x,patch_y=df_in.shape
    temp=[]
    for i in range(no_img):
        i_ary = np.eye(patch_x*patch_y)
        i_ary4d=i_ary[np.newaxis,np.newaxis,:,:]
        weighted_adj=np.zeros((no_ch, no_patch,patch_x*patch_y,patch_x*patch_y))
        df=df_in[i].reshape(no_ch,no_patch,-1)
        pixel1=df[:,:,ed0]
        pixel2=df[:,:,ed1]  
        edu_distance=np.sqrt((pixel1-pixel2)**2)+ 10**(-8)
        weighted_adj[:,:,ed0,ed1]=edu_distance
        sum_r=np.sum(weighted_adj,axis=2)
        temp_adj=sum_r[:,:,:,np.newaxis]
        deg=temp_adj*i_ary4d
        L=deg-weighted_adj
        temp.append(L)
    return temp



In [20]:
from multiprocessing import Pool
import multiprocessing



num_cores = multiprocessing.cpu_count()
print("Number of CPU cores:", num_cores)


Number of CPU cores: 48


In [21]:
train_patch1=np.array_split(reg_train_patch[:], num_cores)
val_patch1=np.array_split(reg_val_patch[:], num_cores)
test_patch1=np.array_split(reg_test_patch[:], num_cores)

In [22]:
print(len(train_patch1))
print(len(val_patch1))
print(len(test_patch1))

48
48
48


In [23]:
num_matrices, num_channels, num_patches, height, width = train_patch1[0].shape

In [24]:
with Pool(processes=num_cores) as pool:
        # Define a list of numbers
    train_p = train_patch1
    
        # Apply the square function to each number using the pool
    g_train=pool.map(compute_grid_lap_channel,train_p)
pool.close()


    
        #results = pool.map(calculate_visibility, numbers)


In [25]:
temp1=[]
for submatrix in g_train:
    temp1.extend(submatrix)


In [26]:
gersh_train= np.array([np.asarray(submatrix) for submatrix in temp1])


In [27]:
print("d")

d


In [28]:
with Pool(processes=num_cores) as pool1:
        # Define a list of numbers
    val_p = val_patch1
    
        # Apply the square function to each number using the pool
    g_val=pool1.map(compute_grid_lap_channel,val_patch1)    
pool1.close()



In [29]:
temp2=[]
for submatrix in g_val:
    temp2.extend(submatrix)


In [30]:
gersh_val= np.array([np.asarray(submatrix) for submatrix in temp2])


In [31]:
print("d")

d


In [32]:
with Pool(processes=num_cores) as pool2:
        # Define a list of numbers
    test_p = test_patch1
    
        # Apply the square function to each number using the pool
    g_test=pool2.map(compute_grid_lap_channel,test_patch1) 
pool2.close()


In [33]:
temp3=[]
for submatrix in g_test:
    temp3.extend(submatrix)


In [34]:
gersh_test= np.array([np.asarray(submatrix) for submatrix in temp3])


In [35]:
print(gersh_train.shape)
print(gersh_val.shape)
print(gersh_test.shape)

(16283, 3, 2500, 4, 4)
(3489, 3, 2500, 4, 4)
(3490, 3, 2500, 4, 4)


In [36]:
lap_train=gersh_train
lap_val=gersh_val
lap_test=gersh_test

In [37]:
print(lap_train.shape)
print(lap_val.shape)
print(lap_test.shape)

(16283, 3, 2500, 4, 4)
(3489, 3, 2500, 4, 4)
(3490, 3, 2500, 4, 4)


In [38]:
# rlap_train=np.moveaxis(lap_train,1,-1)
# rlap_val=np.moveaxis(lap_val,1,-1)
# rlap_test=np.moveaxis(lap_test,1,-1)
rlap_train=lap_train
rlap_val=lap_val
rlap_test=lap_test


In [39]:
print(rlap_train.shape)
print(rlap_val.shape)
print(rlap_test.shape)


(16283, 3, 2500, 4, 4)
(3489, 3, 2500, 4, 4)
(3490, 3, 2500, 4, 4)


In [40]:
rlap_train=rlap_train.reshape(rlap_train.shape[0],rlap_train.shape[1],rlap_train.shape[2]*rlap_train.shape[3],rlap_train.shape[4])
rlap_val=rlap_val.reshape(rlap_val.shape[0],rlap_val.shape[1],rlap_val.shape[2]*rlap_val.shape[3],rlap_val.shape[4])
rlap_test=rlap_test.reshape(rlap_test.shape[0],rlap_test.shape[1],rlap_test.shape[2]*rlap_test.shape[3],rlap_test.shape[4])

In [41]:
print(rlap_train.shape)
print(rlap_val.shape)
print(rlap_test.shape)

(16283, 3, 10000, 4)
(16283, 3, 10000, 4)
(16283, 3, 10000, 4)


In [42]:
rlap_train=rlap_train.reshape(rlap_train.shape[0]*rlap_train.shape[1],-1)
rlap_val=rlap_val.reshape(rlap_val.shape[0]*rlap_val.shape[1],-1)
rlap_test=rlap_test.reshape(rlap_test.shape[0]*rlap_test.shape[1],-1)

In [43]:
print(rlap_train.shape)
print(rlap_val.shape)
print(rlap_test.shape)

(48849, 40000)
(48849, 40000)
(48849, 40000)


In [44]:
L= np.concatenate((rlap_train,rlap_val,rlap_test), axis=0)


In [45]:
print(L.shape)

(69786, 40000)


In [46]:
from sklearn.decomposition import IncrementalPCA
import numpy as np



# Create an instance of IncrementalPCA
n_components = 20000  # Number of components to keep
batch_size = 69786  # Number of samples per batch
ipca = IncrementalPCA(n_components=n_components, batch_size=batch_size)

import time

start_time=time.time()
# Process the dataset in batches
n_batches = L.shape[0] // batch_size
for i in range(n_batches):
    start = i * batch_size
    end = start + batch_size
    batch = L[start:end]
    ipca.partial_fit(batch)
    #print(f"Processed batch {i+1}/{n_batches}")

# Transform the entire dataset using IPCA
transformed_data = ipca.transform(L)
end_time=time.time()

elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

# Check the shape of the transformed data
print(transformed_data.shape)


Elapsed time: 10475.832631349564 seconds
(69786, 20000)


In [47]:
x_train=transformed_data[:lap_train.shape[0]*3]
x_val=transformed_data[lap_train.shape[0]*3:(lap_val.shape[0]*3)+(lap_train.shape[0]*3)]
x_test=transformed_data[(lap_val.shape[0]*3)+(lap_train.shape[0]*3):]

In [48]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(48849, 20000)
(10467, 20000)
(10470, 20000)


In [49]:
x_train=x_train.reshape(lap_train.shape[0],-1)
x_val=x_val.reshape(lap_val.shape[0],-1)
x_test=x_test.reshape(lap_test.shape[0],-1)


In [50]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(16283, 60000)
(3489, 60000)
(3490, 60000)


# IncrementalPCA

In [51]:
input_shape = (x_train.shape[1],1)
num_classes = classes
model2 = tf.keras.models.Sequential()
model2.add(tf.keras.layers.Conv1D(filters= 32, kernel_size=3, activation='relu',padding='same',input_shape= input_shape))
model2.add(tf.keras.layers.AveragePooling1D())
model2.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model2.add(tf.keras.layers.AveragePooling1D())
model2.add(tf.keras.layers.Conv1D(filters=32, kernel_size=3,padding='same',activation='relu'))
model2.add(tf.keras.layers.AveragePooling1D())
model2.add(tf.keras.layers.Flatten())
model2.add(tf.keras.layers.Dense(512, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.1))
model2.add(tf.keras.layers.Dense(512, activation='relu'))
model2.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model2.compile(loss='SparseCategoricalCrossentropy', optimizer='Adam', metrics=['acc'])


In [52]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 60000, 32)         128       
                                                                 
 average_pooling1d (AverageP  (None, 30000, 32)        0         
 ooling1D)                                                       
                                                                 
 conv1d_1 (Conv1D)           (None, 30000, 32)         3104      
                                                                 
 average_pooling1d_1 (Averag  (None, 15000, 32)        0         
 ePooling1D)                                                     
                                                                 
 conv1d_2 (Conv1D)           (None, 15000, 32)         3104      
                                                                 
 average_pooling1d_2 (Averag  (None, 7500, 32)         0

In [53]:
history_log=model2.fit(x_train, reg_train_label, epochs=10,verbose=1,validation_data=(x_val, reg_val_label))


Epoch 1/10
509/509 [==============================] - 238s 462ms/step - loss: 0.6392 - acc: 0.6331 - val_loss: 0.6213 - val_acc: 0.6463
Epoch 2/10
509/509 [==============================] - 236s 464ms/step - loss: 0.5917 - acc: 0.6841 - val_loss: 0.6634 - val_acc: 0.6578
Epoch 3/10
509/509 [==============================] - 235s 462ms/step - loss: 0.5380 - acc: 0.7204 - val_loss: 0.6476 - val_acc: 0.6354
Epoch 4/10
509/509 [==============================] - 233s 457ms/step - loss: 0.4282 - acc: 0.7962 - val_loss: 0.8084 - val_acc: 0.6523
Epoch 5/10
509/509 [==============================] - 240s 472ms/step - loss: 0.2486 - acc: 0.8964 - val_loss: 1.1257 - val_acc: 0.6420
Epoch 6/10
509/509 [==============================] - 255s 501ms/step - loss: 0.1078 - acc: 0.9593 - val_loss: 1.5975 - val_acc: 0.6412
Epoch 7/10
509/509 [==============================] - 255s 501ms/step - loss: 0.0503 - acc: 0.9823 - val_loss: 1.8896 - val_acc: 0.6337
Epoch 8/10
509/509 [============================

In [54]:
loss, accuracy = model2.evaluate(x_test, reg_test_label)
print(f' Model loss on the test set: {loss}')
print(f' Model accuracy on the test set: {100*accuracy}')

110/110 [==============================] - 9s 82ms/step - loss: 2.0807 - acc: 0.6272
 Model loss on the test set: 2.080735921859741
 Model accuracy on the test set: 62.722063064575195


In [55]:
predictions = model2.predict(x_test)
pred_arg = np.argmax(predictions, axis=1)

In [56]:
from sklearn.metrics import confusion_matrix
cnf_matrix=confusion_matrix(reg_test_label,pred_arg)
print("cnf_matrix_2.0_70")
print("")
print(cnf_matrix)

cnf_matrix_2.0_70

[[1039  679]
 [ 622 1150]]


In [57]:
from sklearn.metrics import accuracy_score, recall_score,precision_score,roc_auc_score,roc_curve,f1_score


In [58]:
print(accuracy_score(reg_test_label, pred_arg))
print(f1_score(reg_test_label,pred_arg,average='macro'))
print(recall_score(reg_test_label, pred_arg, average="weighted"))
print(precision_score(reg_test_label, pred_arg, average="weighted"))

0.6272206303724929
0.6268431568324836
0.6272206303724929
0.6271678425982342
